# **Atividade 2 – Perguntas e Respostas usando Modelos do Hugging Face**

### **Dependências**

In [ ]:
!pip install -U --quiet transformers pypdf python-docx pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 131.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 135.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.


In [ ]:
import pypdf
import docx
import pandas as pd
from transformers import pipeline
from transformers.utils import logging
import logging as py_logging
import warnings

### **Texto completo - Doenças Respiratórias Crônicas**

In [ ]:
doencas_cronicas = pypdf.PdfReader("doencas_respiratorias_cronicas.pdf")
conteudo_doencas_cronicas = "\n".join([doencas_cronicas.pages[i].extract_text() for i in range(len(doencas_cronicas.pages))])

print(conteudo_doencas_cronicas[:100])


CADERNOS DE 
ATENÇÃO BÁSICA
MINISTÉRIO DA SAÚDE
DOENÇAS RESPIRATÓRIAS 
CRÔNICAS
Brasília – DF
2010



### **Texto completo - Dicionário de Dados**

In [ ]:
doc = docx.Document("DICIONARIO_DE_DADOS.docx")
textos_completos = []
for para in doc.paragraphs:
    textos_completos.append(para.text)
for table in doc.tables:
    for row in table.rows:
        textos_completos.append(" | ".join(cell.text for cell in row.cells))
dict_dados = "\n".join(textos_completos)


### **Função principal**

In [ ]:
def responder_pergunta(qa_input: dict, model_name: str) -> dict:
    """
    Carrega um modelo de Question Answering e retorna a resposta para uma pergunta.
    Args:
        qa_input (dict): Dicionário com as chaves 'question' e 'context'.
        model_name (str): Nome do modelo de Question Answering a ser carregado.
    Returns:
        dict: Dicionário com as chaves 'answer', 'score' e 'start' e 'end'.
    """
    qa_model = pipeline("question-answering", model=model_name, tokenizer=model_name if model_name != else "unicamp-dl/ptt5-base-portuguese-vocab")
    resultado = qa_model(qa_input)

    return resultado


In [ ]:
modelos = [
    "deepset/xlm-roberta-large-squad2",
    "pierreguillou/bert-base-cased-squad-v1.1-portuguese",
    "benjleite/ptt5-ptbr-qa"
]

In [ ]:
warnings.filterwarnings('ignore')
logging.set_verbosity_error()
py_logging.getLogger("transformers").setLevel(py_logging.ERROR)

### **Doenças respiratórias**

In [ ]:
qa_doencas_respiratorias = [
    "Quais os tratamentos e atos de prevenção para rinite persistente grave?",
    "O que diferencia a hemoptise falsa da verdadeira?",
    "Quais os cuidados ambientais necessários para o controle das crises asmáticas?"
]

In [ ]:
for modelo in modelos:
    for pergunta in qa_doencas_respiratorias:
        qa_input = {
            "question": pergunta,
            "context": conteudo_doencas_cronicas
        }
        resposta = responder_pergunta(qa_input, modelo)
        print(f"Modelo: {modelo}\nPergunta: {pergunta}\nResposta: {resposta}\n\n")

Modelo: deepset/xlm-roberta-large-squad2
Pergunta: Quais os tratamentos e atos de prevenção para rinite persistente grave?
Resposta: {'score': 0.010588130349788116, 'start': 287348, 'end': 287358, 'answer': ' SAMU \n192'}


Modelo: deepset/xlm-roberta-large-squad2
Pergunta: O que diferencia a hemoptise falsa da verdadeira?
Resposta: {'score': 0.010508029728953261, 'start': 25760, 'end': 25774, 'answer': ' o sangramento'}


Modelo: deepset/xlm-roberta-large-squad2
Pergunta: Quais os cuidados ambientais necessários para o controle das crises asmáticas?
Resposta: {'score': 1.8262405736022629, 'start': 68575, 'end': 68618, 'answer': '\nAtividades educativas + controle ambiental'}


Modelo: pierreguillou/bert-base-cased-squad-v1.1-portuguese
Pergunta: Quais os tratamentos e atos de prevenção para rinite persistente grave?
Resposta: {'score': 1.49578197441042, 'start': 212022, 'end': 212037, 'answer': 'Corticosteroids'}


Modelo: pierreguillou/bert-base-cased-squad-v1.1-portuguese
Pergunta: 

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

TypeError: QuestionAnsweringPipeline.__init__() missing 1 required positional argument: 'tokenizer'

In [ ]:
qa_dic_dados = [
    "Quais são os possíveis tipos de serviço referenciado, de acordo com a tabela lfces019?",
    "Que tabelas se relacionam com a tb_carga_horaria_sus?",
    "Recebi um arquivo de planilha via email, que alegava ser uma amostra da tabela rl_estab_serv_class. Ao investigar o arquivo, notei que a planilha apresentava a coluna codservico com alguns registros nulos. Posso considerar esse arquivo como uma amostra verídica?"
]